In [39]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

pd.options.display.max_rows = 128
pd.options.display.max_columns = 128

In [40]:
# Load dataset

df = pd.read_csv('/Users/yc00027/Documents/Project2/petfinder-adoption-prediction/train/train.csv')
print('DF shape:', df.shape)
df.head()

DF shape: (14993, 24)


,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [41]:
# Drop unnecessary columns

to_drop_columns = ['PetID', 'Name', 'RescuerID', 'Description']
df_temp = df.drop(to_drop_columns, axis=1)
print('DF shape:', df_temp.shape)
df_temp.head()

DF shape: (14993, 20)


,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt,AdoptionSpeed
0,2,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,0,1.0,2
1,2,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,0,2.0,0
2,1,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,41326,0,7.0,3
3,1,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,0,8.0,2
4,1,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,0,3.0,2


In [14]:
# Split into train and test sets:

from sklearn.model_selection import train_test_split

X = df_temp.drop(['AdoptionSpeed'], axis=1)
y = df_temp['AdoptionSpeed']

print(X.shape)
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.27, random_state=42)

(14993, 19)
(14993,)


### Logistic Regression ###

In [27]:
# Import necessary modules. Instantiate and fit the model and predict accuracy.

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()

logreg.fit(X_train,y_train)

y_pred_log=logreg.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred_log))

Accuracy: 0.3143986169424549


/Users/yc00027/py3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### Random Forest Classifier ###

In [29]:
# Import necessary modules. Instantiate and fit the model and predict accuracy.

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 10, max_depth=5, random_state = 0)

rf.fit(X_train, y_train)

y_pred_rf_cm = rf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test,y_pred_rf_cm))

Accuracy: 0.38478636700419855


### Random forest with randomized hyperparameter search

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

hyperparameters = {'rt__max_depth': [5, 10, 20, 50],     #     n_estimators: will create the length of the sparse feature.
                   'rt__n_estimators': [1, 10, 25, 50, 75, 100, 200] #     max_depth: will be the maximum depth of the tree
}

pipeline = Pipeline([('rt', RandomForestClassifier())])

clf = RandomizedSearchCV(pipeline, hyperparameters, scoring='accuracy', random_state=1, n_iter=100, cv=5, verbose=0, n_jobs=-1)
clf.fit(X_train, y_train)

print('Best score and parameter combination = ')

print(clf.best_score_)    
print(clf.best_params_)

/Users/yc00027/py3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 28 is smaller than n_iter=100. Running 28 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Best score and parameter combination = 
0.4013155773883545
{'rt__n_estimators': 200, 'rt__max_depth': 20}


In [33]:
clf.best_estimator_

Pipeline(memory=None,
         steps=[('rt',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=20, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=200, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [37]:
y_pred_cm_rscv = clf.best_estimator_.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test,y_pred_cm_rscv))
print("Kappa score:",metrics.cohen_kappa_score(y_test,y_pred_cm_rscv, weights='quadratic'))

Accuracy: 0.4114596196591751
Kappa score: 0.36136269145870525


Kappa Score: Calculates the Cohen Kappa Score or Quadratic Weighted Kappa value, which is a measure of inter-rater agreement between two raters that provide discrete numeric ratings.  Potential values range from -1 (representing complete disagreement) to 1 (representing complete agreement). A kappa value of 0 is expected if all agreement is due to chance.
    
Cohen suggested the Kappa result be interpreted as follows: values ≤ 0 as indicating no agreement and 0.01–0.20 as none to slight, 0.21–0.40 as fair, 0.41– 0.60 as moderate, 0.61–0.80 as substantial, and 0.81–1.00 as almost perfect agreement.

### XGB Classifier

In [74]:
import xgboost as xgb

dtrain = xgb.DMatrix(data=X_train, label=y_train)
dtest = xgb.DMatrix(data=X_test)

/Users/yc00027/py3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [79]:
params = {
    'max_depth': 6,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 5
}

In [80]:
bst = xgb.train(params, dtrain)

In [82]:
y_pred_xgb = bst.predict(dtest)

In [83]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_xgb))
print("Kappa score:",metrics.cohen_kappa_score(y_test, y_pred_xgb, weights='quadratic'))

Accuracy: 0.40083971350950853
Kappa score: 0.3629650232555526


### Support Vector Machine classifier

In [123]:
from sklearn import svm

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score

In [124]:
classifier = svm.SVC(kernel='linear')

In [ ]:
classifier.fit(X_train, y_train) 

In [ ]:
prediction = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, prediction)
print(cm)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test,y_pred_cm_rscv))
print("Kappa score:",metrics.cohen_kappa_score(y_test,y_pred_cm_rscv, weights='quadratic'))